In [346]:
import pandas as pd
import numpy as np
import time as t
from pathlib import Path
from xlwings import view

In [347]:
time_stamp = t.time()

In [348]:
# Source files
file_path_src_pq            = Path("D:/Analytik/3_ФТТ/", "_src_pq.xlsx")
file_path_src_ftt           = Path("D:/Analytik/3_ФТТ/_исх/2025/4/", "фтт.xlsx")
file_path_src_ttzz_squ      = Path("D:/Analytik/3_ФТТ/_исх/2025/4/", "ттзз_скю.xlsx")

# Manuals dataframe
shimpments_qnt_df           = pd.read_excel(file_path_src_pq, "Количество отгрузок")
departments_df              = pd.read_excel(file_path_src_pq, "Справочник отделов")

# Source dataframe
src_ftt_df                  = pd.read_excel(file_path_src_ftt, header=[0])
src_ttzz_squ                = pd.read_excel(file_path_src_ttzz_squ, header=[0])
# Amounting month
month                       = t.strftime("01.04.2025")

In [349]:
src_ftt_df

,Подразделение,СВ_Команда (Пользователи),Код ТА (Пользователи),Менеджер,Клиент,Заказ клиента / Реализация.Дата отгрузки,Выручка+руч.скидка,"Вес (нетто), кг"
0,Беларусская Ферма ОП,БФ Горловка Енакиево,119007.0,Доброштан Владислав Романович,1007.маг.Водолей,19.03.2025,-140.00,-0.800
1,Беларусская Ферма ОП,БФ Горловка Енакиево,119007.0,Доброштан Владислав Романович,1007.маг.Водолей,22.03.2025,-94.00,-0.200
2,Беларусская Ферма ОП,БФ Горловка Енакиево,119007.0,Доброштан Владислав Романович,1007.маг.Водолей,05.04.2025,4851.41,18.302
3,Беларусская Ферма ОП,БФ Горловка Енакиево,119007.0,Доброштан Владислав Романович,1007.маг.Водолей,12.04.2025,6697.67,17.720
4,Беларусская Ферма ОП,БФ Горловка Енакиево,119007.0,Доброштан Владислав Романович,1007.маг.Водолей,23.04.2025,4166.50,11.980
...,...,...,...,...,...,...,...,...
12355,Главмолоко ОП,ГМ Область,118022.0,Горная Анна Викторовна,7460.маг.Продукты 20,24.04.2025,2350.35,15.092
12356,Главмолоко ОП,ГМ Область,118022.0,Горная Анна Викторовна,8982.маг. Никдан,09.04.2025,4865.04,10.710
12357,Главмолоко ОП,ГМ Область,118022.0,Горная Анна Викторовна,8982.маг. Никдан,16.04.2025,2534.15,6.300
12358,Главмолоко ОП,ГМ Область,118022.0,Горная Анна Викторовна,8982.маг. Никдан,30.04.2025,3683.21,6.330


In [350]:
# Количество отгрузок
shimpments_qnt_df = shimpments_qnt_df.loc[shimpments_qnt_df["Начало месяца"] == month]
shimpments_qnt_df.columns = ["Отдел", "Начало месяца", "Значение", "Атрибут"]

# Преобразование object в date
shimpments_qnt_df['Начало месяца'] = pd.to_datetime(shimpments_qnt_df['Начало месяца'], format='%d.%m.%Y')
shimpments_qnt_df = shimpments_qnt_df.set_index(["Отдел", "Начало месяца"])
shimpments_qnt_df

,,Значение,Атрибут
Отдел,Начало месяца,,
Главмолоко,2025-04-01,22,Количество отгрузок
ФЕРМА,2025-04-01,22,Количество отгрузок
VIP,2025-04-01,22,Количество отгрузок
VIP ОПП,2025-04-01,22,Количество отгрузок


In [351]:
df = src_ftt_df.dropna()

# Список полей для переименования
columns = ['Отдел продаж', 'Подразделение', 'Должность ТА', 'ФИО ТА', 'РТТ', 'Дата', 'Значение', 'Вес']
df.columns = columns

# Преобразование object в date
df['Дата'] = pd.to_datetime(df['Дата'], format='%d.%m.%Y')

df = pd.merge(df, departments_df, on=["Отдел продаж", "Подразделение"])
df["Начало месяца"] = pd.to_datetime(df['Дата']).dt.to_period('M').dt.to_timestamp()

<ipython-input-351-9a793afdf373>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Дата'] = pd.to_datetime(df['Дата'], format='%d.%m.%Y')


In [352]:
df["Должность ТА"] = df["Должность ТА"].astype(object)

In [353]:
df = df.drop(columns = ['Вес'])

In [354]:
# Продажи, руб.
sales = df.groupby(["Отдел", "Начало месяца"]).sum()
sales["Атрибут"] = "Продажи, руб."
sales


Значение        Атрибут
Отдел      Начало месяца                            
VIP        2025-04-01      2926144.63  Продажи, руб.
VIP ОПП    2025-04-01       475898.52  Продажи, руб.
Главмолоко 2025-01-01        -3361.12  Продажи, руб.
           2025-02-01       -15041.25  Продажи, руб.
           2025-03-01       -56277.02  Продажи, руб.
           2025-04-01     26639235.17  Продажи, руб.
ФЕРМА      2025-01-01        -1954.04  Продажи, руб.
           2025-02-01        -9629.40  Продажи, руб.
           2025-03-01       -39180.49  Продажи, руб.
           2025-04-01     25387857.77  Продажи, руб.

In [355]:
# Средняя отгрузка
avg_shimpment_df = pd.merge(sales, shimpments_qnt_df, on=["Отдел", "Начало месяца"])
avg_shimpment_df["Значение"] = avg_shimpment_df["Значение_x"] / avg_shimpment_df["Значение_y"]
avg_shimpment_df.drop(columns=["Значение_x", "Значение_y", "Атрибут_x", "Атрибут_y"], inplace=True)
avg_shimpment_df["Атрибут"] = "Средняя отгрузка"
avg_shimpment_df

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2025-04-01,1.330066e+05,Средняя отгрузка
VIP ОПП,2025-04-01,2.163175e+04,Средняя отгрузка
Главмолоко,2025-04-01,1.210874e+06,Средняя отгрузка
ФЕРМА,2025-04-01,1.153994e+06,Средняя отгрузка


In [356]:
# Количество ТРТ
qnt_rtt = df.groupby(["Отдел", "Начало месяца"])["РТТ"].nunique()
qnt_rtt = qnt_rtt.to_frame(name="Значение")
qnt_rtt["Атрибут"] = "Количество ТРТ"
qnt_rtt


Значение         Атрибут
Отдел      Начало месяца                          
VIP        2025-04-01           19  Количество ТРТ
VIP ОПП    2025-04-01           12  Количество ТРТ
Главмолоко 2025-01-01           16  Количество ТРТ
           2025-02-01           62  Количество ТРТ
           2025-03-01          171  Количество ТРТ
           2025-04-01         1426  Количество ТРТ
ФЕРМА      2025-01-01           12  Количество ТРТ
           2025-02-01           37  Количество ТРТ
           2025-03-01          119  Количество ТРТ
           2025-04-01         1466  Количество ТРТ

In [357]:
# Количество должностей
qnt_posts = df.groupby(["Отдел", "Начало месяца"])["Должность ТА"].nunique()
qnt_posts = qnt_posts.to_frame(name="Значение")
qnt_posts["Атрибут"] = "Количество должностей"
qnt_posts

Значение                Атрибут
Отдел      Начало месяца                                 
VIP        2025-04-01            1  Количество должностей
VIP ОПП    2025-04-01            2  Количество должностей
Главмолоко 2025-01-01            8  Количество должностей
           2025-02-01           15  Количество должностей
           2025-03-01           18  Количество должностей
           2025-04-01           23  Количество должностей
ФЕРМА      2025-01-01            7  Количество должностей
           2025-02-01           13  Количество должностей
           2025-03-01           15  Количество должностей
           2025-04-01           19  Количество должностей

In [358]:
# Количество ТА
qnt_ta = df.groupby(["Отдел", "Начало месяца"])["ФИО ТА"].nunique()
qnt_ta = qnt_ta.to_frame(name="Значение")
qnt_ta["Атрибут"] = "Количество ТА"
qnt_ta

Значение        Атрибут
Отдел      Начало месяца                         
VIP        2025-04-01            1  Количество ТА
VIP ОПП    2025-04-01            2  Количество ТА
Главмолоко 2025-01-01            8  Количество ТА
           2025-02-01           15  Количество ТА
           2025-03-01           18  Количество ТА
           2025-04-01           23  Количество ТА
ФЕРМА      2025-01-01            7  Количество ТА
           2025-02-01           13  Количество ТА
           2025-03-01           15  Количество ТА
           2025-04-01           19  Количество ТА

In [359]:
# Количество заказов
qnt_orders = df[["Отдел", "Подразделение", "Начало месяца", "РТТ", "Дата"]].drop_duplicates()
qnt_orders = qnt_orders.groupby(["Отдел", "Начало месяца"])["РТТ"].count()
qnt_orders = qnt_orders.to_frame(name="Значение")
qnt_orders["Атрибут"] = "Количество заказов"
qnt_orders

Значение             Атрибут
Отдел      Начало месяца                              
VIP        2025-04-01          224  Количество заказов
VIP ОПП    2025-04-01           65  Количество заказов
Главмолоко 2025-01-01           17  Количество заказов
           2025-02-01           80  Количество заказов
           2025-03-01          266  Количество заказов
           2025-04-01         6189  Количество заказов
ФЕРМА      2025-01-01           12  Количество заказов
           2025-02-01           46  Количество заказов
           2025-03-01          183  Количество заказов
           2025-04-01         5261  Количество заказов

In [360]:
# Среднее кол-во заказов в ТРТ
avg_orders_qnt = pd.merge(qnt_orders, qnt_rtt, on=["Отдел", "Начало месяца"])
avg_orders_qnt["Значение"] = avg_orders_qnt["Значение_x"]/avg_orders_qnt["Значение_y"]
avg_orders_qnt.drop(columns=["Значение_x", "Атрибут_x", "Значение_y", "Атрибут_y"], inplace=True)
avg_orders_qnt["Атрибут"] = "Среднее кол-во заказов в ТРТ"
avg_orders_qnt

Значение                       Атрибут
Отдел      Начало месяца                                         
VIP        2025-04-01     11.789474  Среднее кол-во заказов в ТРТ
VIP ОПП    2025-04-01      5.416667  Среднее кол-во заказов в ТРТ
Главмолоко 2025-01-01      1.062500  Среднее кол-во заказов в ТРТ
           2025-02-01      1.290323  Среднее кол-во заказов в ТРТ
           2025-03-01      1.555556  Среднее кол-во заказов в ТРТ
           2025-04-01      4.340112  Среднее кол-во заказов в ТРТ
ФЕРМА      2025-01-01      1.000000  Среднее кол-во заказов в ТРТ
           2025-02-01      1.243243  Среднее кол-во заказов в ТРТ
           2025-03-01      1.537815  Среднее кол-во заказов в ТРТ
           2025-04-01      3.588677  Среднее кол-во заказов в ТРТ

In [361]:
# Среднее кол-во ТРТ в день на должность
avq_rtt_day_post = qnt_orders.merge(shimpments_qnt_df, on=["Отдел", "Начало месяца"]).merge(qnt_posts, on=["Отдел", "Начало месяца"])
avq_rtt_day_post["Значение"] = avq_rtt_day_post["Значение_x"]/avq_rtt_day_post["Значение_y"]/avq_rtt_day_post["Значение"]
avq_rtt_day_post.drop(columns=["Значение_x", "Атрибут_x", "Значение_y", "Атрибут_y"], inplace=True)
avq_rtt_day_post["Атрибут"] = "Среднее кол-во ТРТ в день на должность"
avq_rtt_day_post

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2025-04-01,10.181818,Среднее кол-во ТРТ в день на должность
VIP ОПП,2025-04-01,1.477273,Среднее кол-во ТРТ в день на должность
Главмолоко,2025-04-01,12.231225,Среднее кол-во ТРТ в день на должность
ФЕРМА,2025-04-01,12.586124,Среднее кол-во ТРТ в день на должность


In [362]:
# Количество ТРТ с грппировкой по ЗЗ
qnt_rtt_by_orders = df[["Отдел", "Подразделение", "Начало месяца", "РТТ", "Дата"]].drop_duplicates()
qnt_rtt_by_orders = qnt_rtt_by_orders.groupby(["Отдел", "Начало месяца", "РТТ"])["Дата"].count()
qnt_rtt_by_orders = qnt_rtt_by_orders.to_frame(name="Значение")
qnt_rtt_by_orders["Атрибут"] = "Количество ТРТ с грппировкой по ЗЗ"
qnt_rtt_by_orders["Атрибут1"] = np.where(qnt_rtt_by_orders["Значение"] < 3, "1-2 разовых заказов за месяц",
                                  np.where(qnt_rtt_by_orders["Значение"] < 5, "3-4 разовых заказов за месяц",
                                           np.where(qnt_rtt_by_orders["Значение"] < 8, "5-7 разовых заказов за месяц", 
                                                    np.where(qnt_rtt_by_orders["Значение"] >= 8, "8 и выше заказов за месяц", 0)
                                                    )
                                            )
                                  )
qnt_rtt_by_orders = qnt_rtt_by_orders.groupby(["Отдел", "Начало месяца", "Атрибут", "Атрибут1"]).count()
qnt_rtt_by_orders = qnt_rtt_by_orders.reset_index(level=["Атрибут", "Атрибут1"])
qnt_rtt_by_orders

Атрибут  \
Отдел      Начало месяца                                       
VIP        2025-04-01     Количество ТРТ с грппировкой по ЗЗ   
           2025-04-01     Количество ТРТ с грппировкой по ЗЗ   
VIP ОПП    2025-04-01     Количество ТРТ с грппировкой по ЗЗ   
           2025-04-01     Количество ТРТ с грппировкой по ЗЗ   
           2025-04-01     Количество ТРТ с грппировкой по ЗЗ   
           2025-04-01     Количество ТРТ с грппировкой по ЗЗ   
Главмолоко 2025-01-01     Количество ТРТ с грппировкой по ЗЗ   
           2025-02-01     Количество ТРТ с грппировкой по ЗЗ   
           2025-02-01     Количество ТРТ с грппировкой по ЗЗ   
           2025-03-01     Количество ТРТ с грппировкой по ЗЗ   
           2025-03-01     Количество ТРТ с грппировкой по ЗЗ   
           2025-03-01     Количество ТРТ с грппировкой по ЗЗ   
           2025-04-01     Количество ТРТ с грппировкой по ЗЗ   
           2025-04-01     Количество ТРТ с грппировкой по ЗЗ   
           2025-04-01     Количество ТРТ с грппировкой по ЗЗ   
           2025-04-01     Количество ТРТ с грппировкой по ЗЗ   
ФЕРМА      2025-01-01     Количество ТРТ с грппировкой по ЗЗ   
           2025-02-01     Количество ТРТ с грппировкой по ЗЗ   
           2025-02-01     Количество ТРТ с грппировкой по ЗЗ   
           2025-03-01     Количество ТРТ с грппировкой по ЗЗ   
           2025-03-01     Количество ТРТ с грппировкой по ЗЗ   
           2025-04-01     Количество ТРТ с грппировкой по ЗЗ   
           2025-04-01     Количество ТРТ с грппировкой по ЗЗ   
           2025-04-01     Количество ТРТ с грппировкой по ЗЗ   
           2025-04-01     Количество ТРТ с грппировкой по ЗЗ   

                                              Атрибут1  Значение  
Отдел      Начало месяца                                          
VIP        2025-04-01     1-2 разовых заказов за месяц         1  
           2025-04-01        8 и выше заказов за месяц        18  
VIP ОПП    2025-04-01     1-2 разовых заказов за месяц         4  
           2025-04-01     3-4 разовых заказов за месяц         2  
           2025-04-01     5-7 разовых заказов за месяц         1  
           2025-04-01        8 и выше заказов за месяц         5  
Главмолоко 2025-01-01     1-2 разовых заказов за месяц        16  
           2025-02-01     1-2 разовых заказов за месяц        60  
           2025-02-01     3-4 разовых заказов за месяц         2  
           2025-03-01     1-2 разовых заказов за месяц       154  
           2025-03-01     3-4 разовых заказов за месяц        16  
           2025-03-01     5-7 разовых заказов за месяц         1  
           2025-04-01     1-2 разовых заказов за месяц       400  
           2025-04-01     3-4 разовых заказов за месяц       484  
           2025-04-01     5-7 разовых заказов за месяц       368  
           2025-04-01        8 и выше заказов за месяц       174  
ФЕРМА      2025-01-01     1-2 разовых заказов за месяц        12  
           2025-02-01     1-2 разовых заказов за месяц        36  
           2025-02-01     3-4 разовых заказов за месяц         1  
           2025-03-01     1-2 разовых заказов за месяц       103  
           2025-03-01     3-4 разовых заказов за месяц        16  
           2025-04-01     1-2 разовых заказов за месяц       625  
           2025-04-01     3-4 разовых заказов за месяц       490  
           2025-04-01     5-7 разовых заказов за месяц       219  
           2025-04-01        8 и выше заказов за месяц       132

In [363]:
# Средний заказ, руб.
avg_order = sales.merge(qnt_orders, on=["Отдел", "Начало месяца"])
avg_order["Значение"] = avg_order["Значение_x"]/avg_order["Значение_y"]
avg_order.drop(columns=["Значение_x", "Атрибут_x", "Значение_y", "Атрибут_y"], inplace=True)
avg_order["Атрибут"] = "Средний заказ, руб."
avg_order

Значение              Атрибут
Отдел      Начало месяца                                   
VIP        2025-04-01     13063.145670  Средний заказ, руб.
VIP ОПП    2025-04-01      7321.515692  Средний заказ, руб.
Главмолоко 2025-01-01      -197.712941  Средний заказ, руб.
           2025-02-01      -188.015625  Средний заказ, руб.
           2025-03-01      -211.567744  Средний заказ, руб.
           2025-04-01      4304.287473  Средний заказ, руб.
ФЕРМА      2025-01-01      -162.836667  Средний заказ, руб.
           2025-02-01      -209.334783  Средний заказ, руб.
           2025-03-01      -214.101038  Средний заказ, руб.
           2025-04-01      4825.671502  Средний заказ, руб.

In [364]:
# Средняя продажа в ТРТ, руб
avg_rtt_sale = sales.merge(qnt_rtt, on=["Отдел", "Начало месяца"])
avg_rtt_sale["Значение"] = avg_rtt_sale["Значение_x"]/avg_rtt_sale["Значение_y"]
avg_rtt_sale.drop(columns=["Значение_x", "Атрибут_x", "Значение_y", "Атрибут_y"], inplace=True)
avg_rtt_sale["Атрибут"] = "Средняя продажа в ТРТ, руб"
avg_rtt_sale

Значение                     Атрибут
Отдел      Начало месяца                                           
VIP        2025-04-01     154007.612105  Средняя продажа в ТРТ, руб
VIP ОПП    2025-04-01      39658.210000  Средняя продажа в ТРТ, руб
Главмолоко 2025-01-01       -210.070000  Средняя продажа в ТРТ, руб
           2025-02-01       -242.600806  Средняя продажа в ТРТ, руб
           2025-03-01       -329.105380  Средняя продажа в ТРТ, руб
           2025-04-01      18681.090582  Средняя продажа в ТРТ, руб
ФЕРМА      2025-01-01       -162.836667  Средняя продажа в ТРТ, руб
           2025-02-01       -260.254054  Средняя продажа в ТРТ, руб
           2025-03-01       -329.247815  Средняя продажа в ТРТ, руб
           2025-04-01      17317.774741  Средняя продажа в ТРТ, руб

In [365]:
src_ttzz_squ.head(1)

,Менеджер.Подразделение,Менеджер.СВ_Команда (Пользователи),Товарная категория,Марка (бренд),Тип товара производителя (Общие),Артикул,Номенклатура,Выручка+руч.скидка,"Вес (нетто), кг"
0,Беларусская Ферма ОП,БФ Горловка Енакиево,Десерты Молочно-Творожные,Без ТМ ТП,Без ТМ Крем смет 3кг,14243.0,Крем По Технологии Сметаны Клубника 13% Без ТМ...,2501.94,9.0


In [366]:
# Активный ассортимент
df2 = src_ttzz_squ.dropna()

# Преобразование object в date
df2['Начало месяца'] = pd.to_datetime(month, format='%d.%m.%Y')

# Список полей для переименования
columns = ['Отдел продаж', 'Подразделение',	'Тип товара дистрибьютора',	'Торговая марка',	'Тип товара производителя',	'Код ТМЦ',	'ТМЦ', 'Значение', 'Вес', 'Начало месяца']
df2.columns = columns
df2.drop(columns='Вес')

df2 = pd.merge(df2, departments_df, on=["Отдел продаж", "Подразделение"])

active_assort = df2.groupby(["Отдел", "Начало месяца", "Торговая марка"])["ТМЦ"].nunique()
active_assort = active_assort.to_frame("Значение")
active_assort["Атрибут"] = "Активный ассортимент"
active_assort = active_assort.rename_axis(index={"Торговая марка" : "Атрибут1"})
active_assort = active_assort.reset_index(level= "Атрибут1")
active_assort = active_assort[["Значение", "Атрибут", "Атрибут1"]]
active_assort

<ipython-input-366-aad878bad160>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Начало месяца'] = pd.to_datetime(month, format='%d.%m.%Y')


Значение               Атрибут              Атрибут1
Отдел      Начало месяца                                                      
VIP        2025-04-01            5  Активный ассортимент             Без ТМ ТП
           2025-04-01           16  Активный ассортимент             Белавушка
           2025-04-01           18  Активный ассортимент            Главмолоко
           2025-04-01           32  Активный ассортимент        Живая Марка ТП
           2025-04-01            7  Активный ассортимент              Живая ТП
           2025-04-01           17  Активный ассортимент                 ФЕРМА
VIP ОПП    2025-04-01            7  Активный ассортимент             Без ТМ ТП
           2025-04-01            6  Активный ассортимент             Белавушка
           2025-04-01           17  Активный ассортимент            Главмолоко
           2025-04-01           31  Активный ассортимент        Живая Марка ТП
           2025-04-01            2  Активный ассортимент              Живая ТП
           2025-04-01            2  Активный ассортимент         Ретро Вкус ТП
           2025-04-01            4  Активный ассортимент                 ФЕРМА
Главмолоко 2025-04-01            2  Активный ассортимент                 Bodro
           2025-04-01            2  Активный ассортимент            Bodro Milk
           2025-04-01           12  Активный ассортимент          Grand Tomato
           2025-04-01            1  Активный ассортимент        Grands Gорчица
           2025-04-01            8  Активный ассортимент           Uno Momento
           2025-04-01            1  Активный ассортимент            Без ТМ КПП
           2025-04-01           17  Активный ассортимент             Без ТМ ТП
           2025-04-01            3  Активный ассортимент             Белавушка
           2025-04-01            3  Активный ассортимент  Белорусский Стандарт
           2025-04-01            5  Активный ассортимент         Вершина Вкуса
           2025-04-01           24  Активный ассортимент            Главмолоко
           2025-04-01           38  Активный ассортимент        Живая Марка ТП
           2025-04-01           13  Активный ассортимент              Живая ТП
           2025-04-01           13  Активный ассортимент               Калория
           2025-04-01            2  Активный ассортимент                 Норма
           2025-04-01            1  Активный ассортимент      Премиум Стандарт
           2025-04-01            4  Активный ассортимент            Ясное Утро
ФЕРМА      2025-04-01            2  Активный ассортимент                 Bodro
           2025-04-01            2  Активный ассортимент            Bodro Milk
           2025-04-01           12  Активный ассортимент          Grand Tomato
           2025-04-01            1  Активный ассортимент        Grands Gорчица
           2025-04-01            8  Активный ассортимент           Uno Momento
           2025-04-01            2  Активный ассортимент            Без ТМ КПП
           2025-04-01           17  Активный ассортимент             Без ТМ ТП
           2025-04-01           39  Активный ассортимент             Белавушка
           2025-04-01            5  Активный ассортимент         Вершина Вкуса
           2025-04-01            8  Активный ассортимент        Живая Марка ТП
           2025-04-01            5  Активный ассортимент              Живая ТП
           2025-04-01           13  Активный ассортимент               Калория
           2025-04-01            2  Активный ассортимент                 Норма
           2025-04-01            3  Активный ассортимент                 Ретро
           2025-04-01            4  Активный ассортимент         Ретро Вкус ТП
           2025-04-01           32  Активный ассортимент                 ФЕРМА

In [367]:
ftt_df = pd.concat([active_assort
                    , avg_rtt_sale
                    , avg_order
                    , qnt_rtt_by_orders
                    , avq_rtt_day_post
                    , avg_orders_qnt
                    , qnt_orders
                    , qnt_ta
                    , qnt_posts
                    , qnt_rtt
                    , avg_shimpment_df
                    , sales
                    , shimpments_qnt_df
                    ])

# Filter by month
view(ftt_df.loc[[("VIP", pd.to_datetime(month, format='%d.%m.%Y')) 
            , ("VIP ОПП", pd.to_datetime(month, format='%d.%m.%Y'))
            , ("Главмолоко", pd.to_datetime(month, format='%d.%m.%Y'))
            , ("ФЕРМА", pd.to_datetime(month, format='%d.%m.%Y'))
            ]]
)


In [368]:
t.time() - time_stamp 

3.7229061126708984